In [132]:
import pandas as pd
import os
import re
import numpy as np
from pathlib import Path
from tqdm.auto import tqdm
import scipy.stats

In [216]:
OUTPUT_DIR = "/h/haoran/projects/HurtfulWords/data/models/finetuned/debiasing_finetuning_10.0/baseline_clinical_BERT_1_epoch_512/phenotype_first/"
# OUTPUT_DIR = "/h/haoran/projects/HurtfulWords/data/models/finetuned/debiasing_finetuning_1.0/baseline_clinical_BERT_1_epoch_512/phenotype_first/"
# OUTPUT_DIR = "/h/haoran/projects/HurtfulWords/data/models/finetuned/baseline_clinical_BERT_1_epoch_512/phenotype_first/"
HIDE_CIs = True
SIG = 0.95

In [217]:
sheets = ['all', 'gender' ,'language_to_use', 'insurance', 'ethnicity_to_use'] 

In [218]:
def gap_significant(lower, upper):
    return -1*((lower < 0) & (upper < 0)) + ((lower >0) & (upper > 0))

In [219]:
dfs = {i: [] for i in sheets}
for subdir, dirs, files in tqdm(os.walk(OUTPUT_DIR)):
    for split in dirs:
        for sheet in sheets:
            base_dir = os.path.join(OUTPUT_DIR, split)
            file_name = os.path.join(base_dir, 'results.xlsx')
            df = pd.read_excel(file_name, index_col=0, sheet_name = sheet)
            dfs[sheet].append(df)          
        

In [220]:
def conf(mat):
    return scipy.stats.t.interval(SIG, mat.shape[0]-1, loc=np.mean(mat, axis= 0), scale=scipy.stats.sem(mat, axis = 0))

In [221]:
res = pd.DataFrame(index = dfs['all'][0].index)
for sheet in sheets:    
    if sheet == 'all':
        columns = ['all_auroc', 'all_auprc','all_specificity', 'all_recall']        
        for i in columns:
            temp = []
            for split in dfs[sheet]:
                temp.append(split[i])
            res[i] = np.mean(np.array(temp), axis = 0)
            res[i+ '_std'] = np.std(np.array(temp), axis = 0)

    elif sheet == 'gender':
        columns = {
                'gender=="F"_dgap_max':'Parity Gap (F-M)',
                   'gender=="F"_egap_positive_max':'Recall Gap',
                   'gender=="F"_egap_negative_max':'Specificity Gap',
                  }
        for i in columns:
            n = i.replace('gender==', '')
            temp = []
            for split in dfs[sheet]:
                temp.append(split[i])
            res[n] = np.mean(np.array(temp), axis = 0)
            lower, upper = conf(np.array(temp))
            res[n + '_lower'] = lower
            res[n + '_upper'] = upper            
            res[n + '_sig'] = gap_significant(lower, upper)                 

    elif sheet == 'insurance':                            
        columns = []
        for i in ['Medicare', 'Private', 'Medicaid']:
#             for j in ['precision', 'recall', 'specificity', 'pred_prevalence', 'actual_prevalence',
#                      'nsamples', 'dgap_max', 'egap_positive_max', 'egap_negative_max']:
            for j in ['dgap_max', 'egap_positive_max', 'egap_negative_max']:
                columns.append(
                    'insurance=="%s"_%s'%(i,j)
                )
        for i in columns:
            temp = []
            n = i.replace('insurance==', '')
            for split in dfs[sheet]:
                temp.append(split[i])
            res[n] = np.mean(np.array(temp), axis = 0)
            lower, upper = conf(np.array(temp))
            res[n + '_lower'] = lower
            res[n + '_upper'] = upper            
            res[n + '_sig'] = gap_significant(lower, upper)      

    elif sheet == 'language_to_use':
        columns = {'language_to_use=="English"_dgap_max' : 'Parity Gap (E-O)', 
                   'language_to_use=="English"_egap_positive_max' : 'Recall Gap', 
                   'language_to_use=="English"_egap_negative_max' : 'Specificity Gap' }
        for i in columns:
            temp = []
            n = i.replace('language_to_use==', '')
            for split in dfs[sheet]:
                temp.append(split[i])
            res[n] = np.mean(np.array(temp), axis = 0)
            lower, upper = conf(np.array(temp))
            res[n + '_lower'] = lower
            res[n + '_upper'] = upper            
            res[n + '_sig'] = gap_significant(lower, upper)      

    elif sheet == 'ethnicity_to_use':
        columns = []
        for i in ['WHITE', 'BLACK', 'ASIAN', 'HISPANIC/LATINO', 'OTHER']:
#             for j in ['precision', 'recall', 'specificity', 'pred_prevalence', 'actual_prevalence',
#                      'nsamples', 'dgap_max', 'egap_positive_max', 'egap_negative_max']:
            for j in ['dgap_max', 'egap_positive_max', 'egap_negative_max']:
                columns.append(
                    'ethnicity_to_use=="%s"_%s'%(i,j)
                )
        for i in columns:
            temp = []
            n = i.replace('ethnicity_to_use==', '')
            for split in dfs[sheet]:
                temp.append(split[i])
            res[n] = np.mean(np.array(temp), axis = 0)
            lower, upper = conf(np.array(temp))
            res[n + '_lower'] = lower
            res[n + '_upper'] = upper            
            res[n + '_sig'] = gap_significant(lower, upper)      
        
#     dfs[sheet] = res.sort_index()

/h/haoran/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1910: RuntimeWarning: invalid value encountered in multiply
  lower_bound = self.a * scale + loc
/h/haoran/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1911: RuntimeWarning: invalid value encountered in multiply
  upper_bound = self.b * scale + loc
/h/haoran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in less
  
/h/haoran/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in greater
  


In [222]:
data = []
for i in res.columns:
    if 'sig' in i and 'egap_positive' in i:
        negative = (res[i] == -1).sum()
        positive = (res[i] == 1).sum()
        data.append([i,negative+ positive, positive])
pd.DataFrame(data, columns = ['type','Total Sig','Total For'])

,type,Total Sig,Total For
0,"""F""_egap_positive_max_sig",14,11
1,"""English""_egap_positive_max_sig",13,1
2,"""Medicare""_egap_positive_max_sig",23,22
3,"""Private""_egap_positive_max_sig",14,0
4,"""Medicaid""_egap_positive_max_sig",25,2
5,"""WHITE""_egap_positive_max_sig",10,9
6,"""BLACK""_egap_positive_max_sig",3,0
7,"""ASIAN""_egap_positive_max_sig",5,5
8,"""HISPANIC/LATINO""_egap_positive_max_sig",7,1
9,"""OTHER""_egap_positive_max_sig",1,0


In [223]:
res['all_auroc'].mean()

0.6762327490370332